In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:

df = pd.read_csv("diabetes.csv")
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
df.corr()["Outcome"]

,Outcome
Pregnancies,0.221898
Glucose,0.466581
BloodPressure,0.065068
SkinThickness,0.074752
Insulin,0.130548
BMI,0.292695
DiabetesPedigreeFunction,0.173844
Age,0.238356
Outcome,1.000000


In [4]:
X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [5]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)

In [6]:
X

array([[ 0.63994726,  0.84832379,  0.14964075, ...,  0.20401277,
         0.46849198,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.16054575, ..., -0.68442195,
        -0.36506078, -0.19067191],
       [ 1.23388019,  1.94372388, -0.26394125, ..., -1.10325546,
         0.60439732, -0.10558415],
       ...,
       [ 0.3429808 ,  0.00330087,  0.14964075, ..., -0.73518964,
        -0.68519336, -0.27575966],
       [-0.84488505,  0.1597866 , -0.47073225, ..., -0.24020459,
        -0.37110101,  1.17073215],
       [-0.84488505, -0.8730192 ,  0.04624525, ..., -0.20212881,
        -0.47378505, -0.87137393]])

In [7]:
X.shape

(768, 8)

In [8]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [9]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense

In [10]:
model = Sequential()
model.add(Dense(32,activation="relu",input_dim=8))
model.add(Dense(1, activation="sigmoid"))
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [11]:
model.fit(X_train,y_train,batch_size=32,epochs=10,validation_data=(X_test,y_test))

Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.5220 - loss: 0.7013 - val_accuracy: 0.6364 - val_loss: 0.6374
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6607 - loss: 0.6222 - val_accuracy: 0.7078 - val_loss: 0.5895
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7111 - loss: 0.5957 - val_accuracy: 0.7403 - val_loss: 0.5526
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7155 - loss: 0.5654 - val_accuracy: 0.7662 - val_loss: 0.5226
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7398 - loss: 0.5332 - val_accuracy: 0.7727 - val_loss: 0.5003
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7698 - loss: 0.5060 - val_accuracy: 0.7792 - val_loss: 0.4842
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7733 - loss: 0.5018 - val_accuracy: 0.7792 - val_loss: 0.4730
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7975 - loss: 0.4717 - val_accuracy: 0.7857 - val_loss

In [12]:
pip install -U keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 4.3 MB/s eta 0:00:00


In [13]:
import kerastuner as kt

/tmp/ipython-input-1654478174.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [14]:
def build_model(hp):
    model = Sequential()
    model.add(Dense(32,activation = "relu", input_dim = 8))
    model.add(Dense(1,activation = "sigmoid"))
    optimizer = hp.Choice("optimizer", values = ["adam","sgd","rmsprop","adadelta"])
    model.compile(optimizer = optimizer, loss = "binary_crossentropy", metrics = ["accuracy"])
    return model


In [15]:
tuner = kt.RandomSearch(build_model, objective = "val_accuracy", max_trials = 5)

In [16]:
tuner.search(X_train,y_train, epochs = 5, validation_data = (X_test,y_test))

Trial 4 Complete [00h 00m 06s]
val_accuracy: 0.7727272510528564

Best val_accuracy So Far: 0.7792207598686218
Total elapsed time: 00h 00m 20s


In [17]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [18]:
model = tuner.get_best_models(num_models = 1)[0]

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [20]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [22]:
model.fit(X_train,y_train,batch_size=32,epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7474 - loss: 0.5154 - val_accuracy: 0.8182 - val_loss: 0.4560
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7357 - loss: 0.5195 - val_accuracy: 0.8182 - val_loss: 0.4513
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7353 - loss: 0.5318 - val_accuracy: 0.8117 - val_loss: 0.4475
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7682 - loss: 0.4944 - val_accuracy: 0.8117 - val_loss: 0.4454
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7853 - loss: 0.4639 - val_accuracy: 0.8182 - val_loss: 0.4444
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7699 - loss: 0.4873 - val_accuracy: 0.8247 - val_loss: 0.4422
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7582 - loss: 0.4813 - val_accuracy: 0.8312 - val_loss: 0.4419
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7526 - loss: 0.4860 - val_accuracy: 0.831

In [32]:
def build_model(hp):
  model = Sequential()
  units = hp.Int(('units'),min_value = 8,max_value = 128,step=8)
  model.add(Dense(units=units, activation="relu", input_dim=8))
  model.add(Dense(1,activation="sigmoid"))
  model.compile(optimizer="rmsprop",loss = "binary_crossentropy",metrics=["accuracy"])
  return model

In [33]:
tuner = kt.RandomSearch(build_model,objective="val_accuracy",max_trials=5,directory = "mydir")

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [34]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.7727272510528564

Best val_accuracy So Far: 0.8181818127632141
Total elapsed time: 00h 00m 13s


In [35]:
tuner.get_best_hyperparameters()[0].values

{'units': 80}

In [36]:
model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [37]:
model.fit(X_train,y_train,batch_size=32,epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.7506 - loss: 0.4988 - val_accuracy: 0.8182 - val_loss: 0.4454
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7655 - loss: 0.4874 - val_accuracy: 0.8182 - val_loss: 0.4388
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7915 - loss: 0.4547 - val_accuracy: 0.8312 - val_loss: 0.4346
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7838 - loss: 0.4635 - val_accuracy: 0.8312 - val_loss: 0.4313
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8086 - loss: 0.4307 - val_accuracy: 0.8312 - val_loss: 0.4312
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7962 - loss: 0.4411 - val_accuracy: 0.8247 - val_loss: 0.4305
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7548 - loss: 0.4618 - val_accuracy: 0.8312 - val_loss: 0.4304
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7583 - loss: 0.4657 - val_accuracy: 0.82

In [40]:
def build_model(hp):
  model = Sequential()
  model.add(Dense(72, activation = "relu",input_dim = 8))
  for i in range(hp.Int("num_layers",min_value=1,max_value=10)):
    model.add(Dense(72, activation = "relu"))
  model.add(Dense(1, activation = "sigmoid"))
  model.compile(optimizer='rmsprop' , loss = "binary_crossentropy", metrics = ["accuracy"])
  return model


In [41]:
tuner  = kt.RandomSearch(build_model, objective = "val_accuracy", max_trials = 3,directory = "mydir",project_name = "numLayers")

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [42]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 3 Complete [00h 00m 04s]
val_accuracy: 0.7857142686843872

Best val_accuracy So Far: 0.8246753215789795
Total elapsed time: 00h 00m 11s


In [43]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 2}

In [48]:
model = tuner.get_best_models(num_models = 1)[0]

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [49]:
model.fit(X_train,y_train,batch_size=32,epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.7485 - loss: 0.5050 - val_accuracy: 0.8182 - val_loss: 0.4295
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7765 - loss: 0.4627 - val_accuracy: 0.7987 - val_loss: 0.4370
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7821 - loss: 0.4686 - val_accuracy: 0.7987 - val_loss: 0.4392
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8100 - loss: 0.4177 - val_accuracy: 0.8247 - val_loss: 0.4355
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8082 - loss: 0.4153 - val_accuracy: 0.7922 - val_loss: 0.4450
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7984 - loss: 0.4239 - val_accuracy: 0.7857 - val_loss: 0.4562
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8055 - loss: 0.4288 - val_accuracy: 0.8182 - val_loss: 0.4394
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7838 - loss: 0.4163 - val_accuracy: 0.81

In [54]:
def build_model(hp):
  model = Sequential()
  counter = 0
  for i in range(hp.Int("num_layers",min_value = 1 , max_value = 10)):
    if counter == 0:
      model.add(
          Dense(
              units = hp.Int("units"+str(i),min_value = 8 , max_value = 128 , step = 8),
              activation = hp.Choice("activation"+str(i),values = ["relu","tanh","sigmoid"]),
              input_dim = 8
          )
      )
    else:
      model.add(
          Dense(
              units = hp.Int("units"+str(i),min_value = 8 , max_value = 128 , step = 8),
              activation = hp.Choice("activation"+str(i),values = ["relu","tanh","sigmoid"]),
          )
      )
    counter +=1
    model.add(Dense(1,activation = "sigmoid"))
    model.compile(optimizer = hp.Choice("optimizer",values = ["adam","sgd","rmsprop","adadelta"]),loss = "binary_crossentropy",metrics = ["accuracy"])
    return model


In [55]:
tuner = kt.RandomSearch(build_model,objective = "val_accuracy",max_trials = 3,directory = "mydir",project_name = "activation")

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [56]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 3 Complete [00h 00m 03s]
val_accuracy: 0.8181818127632141

Best val_accuracy So Far: 0.8181818127632141
Total elapsed time: 00h 00m 10s


In [57]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 7, 'units0': 88, 'activation0': 'tanh', 'optimizer': 'rmsprop'}

In [58]:
model = tuner.get_best_models(num_models = 1)[0]

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [60]:
model.fit(X_train,y_train,batch_size=32,epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.7496 - loss: 0.5132 - val_accuracy: 0.7987 - val_loss: 0.4379
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7541 - loss: 0.4923 - val_accuracy: 0.8117 - val_loss: 0.4315
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7999 - loss: 0.4444 - val_accuracy: 0.8182 - val_loss: 0.4351
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7631 - loss: 0.4769 - val_accuracy: 0.8182 - val_loss: 0.4333
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7671 - loss: 0.4890 - val_accuracy: 0.7987 - val_loss: 0.4358
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7620 - loss: 0.4892 - val_accuracy: 0.8052 - val_loss: 0.4347
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7591 - loss: 0.4672 - val_accuracy: 0.8117 - val_loss: 0.4330
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7578 - loss: 0.4831 - val_accuracy: 0.81